In [1]:
from keras.models import Model
from keras.layers import Activation, Input, concatenate, Dropout, Dense, Flatten
from keras.layers import MaxPooling2D, Conv2D, AveragePooling2D, BatchNormalization
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

def block_stem(inputs):
    net = conv2d(inputs, 32, (3, 3), strides=(2, 2), padding='valid')
    net = conv2d(net, 32, (3, 3), padding='valid')
    net = conv2d(net, 64, (3, 3))

    branch_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(net)
    branch_2 = conv2d(net, 96, (3, 3), strides=(2, 2), padding='valid')

    net = concatenate([branch_1, branch_2])

    branch_1 = conv2d(net, 64, (1, 1))
    branch_1 = conv2d(branch_1, 96, (3, 3), padding='valid')

    branch_2 = conv2d(net, 64, (1, 1))
    branch_2 = conv2d(branch_2, 64, (7, 1))
    branch_2 = conv2d(branch_2, 64, (1, 7))
    branch_2 = conv2d(branch_2, 96, (3, 3), padding='valid')

    net = concatenate([branch_1, branch_2])

    branch_1 = conv2d(net, 192, (3, 3), strides=(2, 2), padding='valid')  # different from the paper
    branch_2 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(net)

    net = concatenate([branch_1, branch_2])

    return net


def block_inception_a(inputs):
    branch_1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 96, (1, 1))

    branch_2 = conv2d(inputs, 96, (1, 1))

    branch_3 = conv2d(inputs, 64, (1, 1))
    branch_3 = conv2d(branch_3, 96, (3, 3))

    branch_4 = conv2d(inputs, 64, (1, 1))
    branch_4 = conv2d(branch_4, 96, (3, 3))
    branch_4 = conv2d(branch_4, 96, (3, 3))

    return concatenate([branch_1, branch_2, branch_3, branch_4])


def block_inception_b(inputs):
    branch_1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 128, (1, 1))

    branch_2 = conv2d(inputs, 384, (1, 1))

    branch_3 = conv2d(inputs, 192, (1, 1))
    branch_3 = conv2d(branch_3, 224, (1, 7))
    branch_3 = conv2d(branch_3, 256, (7, 1))  # different from the paper

    branch_4 = conv2d(inputs, 192, (1, 1))
    branch_4 = conv2d(branch_4, 192, (1, 7))
    branch_4 = conv2d(branch_4, 224, (7, 1))
    branch_4 = conv2d(branch_4, 224, (1, 7))
    branch_4 = conv2d(branch_4, 256, (7, 1))

    return concatenate([branch_1, branch_2, branch_3, branch_4])


def block_inception_c(inputs):
    branch_1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 256, (1, 1))

    branch_2 = conv2d(inputs, 256, (1, 1))

    branch_3 = conv2d(inputs, 384, (1, 1))
    branch_3_1 = conv2d(branch_3, 256, (1, 3))
    branch_3_2 = conv2d(branch_3, 256, (3, 1))

    branch_4 = conv2d(inputs, 384, (1, 1))
    branch_4 = conv2d(branch_4, 448, (1, 3))
    branch_4 = conv2d(branch_4, 512, (3, 1))
    branch_4_1 = conv2d(branch_4, 256, (3, 1))
    branch_4_2 = conv2d(branch_4, 256, (1, 3))

    return concatenate([branch_1, branch_2, branch_3_1, branch_3_2, branch_4_1, branch_4_2])


def block_reduction_a(inputs):
    branch_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(inputs)

    branch_2 = conv2d(inputs, 384, (3, 3), strides=(2, 2), padding='valid')

    branch_3 = conv2d(inputs, 192, (1, 1))
    branch_3 = conv2d(branch_3, 224, (3, 3))
    branch_3 = conv2d(branch_3, 256, (3, 3), strides=(2, 2), padding='valid')

    return concatenate([branch_1, branch_2, branch_3])


def block_reduction_b(inputs):
    branch_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(inputs)

    branch_2 = conv2d(inputs, 192, (1, 1))
    branch_2 = conv2d(branch_2, 192, (3, 3), strides=(2, 2), padding='valid')

    branch_3 = conv2d(inputs, 256, (1, 1))
    branch_3 = conv2d(branch_3, 256, (1, 7))
    branch_3 = conv2d(branch_3, 320, (7, 1))
    branch_3 = conv2d(branch_3, 320, (3, 3), strides=(2, 2), padding='valid')

    return concatenate([branch_1, branch_2, branch_3])


def conv2d(net, filters, kernel_size, strides=(1, 1), padding='same'):
    net = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, use_bias=False)(net)
    net = BatchNormalization()(net)
    net = Activation('relu')(net)
    return net

def create(classes_num=4):
    inputs = Input((227, 227, 3))

    # 227 x 227 x 3
    net = block_stem(inputs)

    # 4 x Inception-A ( Output: 35 x 35 x 384 )
    for i in range(4):
        net = block_inception_a(net)

    # Reduction-A ( Output: 17 x 17 x 1024 )
    net = block_reduction_a(net)

    # 7 x Inception-B ( Output: 17 x 17 x 1024 )
    for i in range(7):
        net = block_inception_b(net)

    # Reduction-B ( Output: 8 x 8 x 1536 )
    net = block_reduction_b(net)

    # 3 x Inception-C ( Output: 8 x 8 x 1536 )
    for i in range(3):
        net = block_inception_c(net)

    # Average Pooling ( Output: 1536 )
    net = AveragePooling2D((5, 5))(net)

    # Dropout ( keep 0.8 )
    net = Dropout(0.2)(net)
    net = Flatten()(net)

    # Output
    outputs = Dense(units=classes_num, activation='softmax')(net)

    return Model(inputs, outputs, name='Inception-v4')

In [2]:
def inceptionV4_model(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # validation split

    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.1, random_state=10,
                                                                    stratify=y_train)
    print(len(X_train))


    model = create()
    model.compile(loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'],
                  optimizer=keras.optimizers.SGD(learning_rate=0.01))
    model.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_validation, y_validation))
    
    evaluation = []
    evaluation.append(model.evaluate(X_train, y_train, batch_size=16))
    evaluation.append(model.evaluate(X_test, y_test, batch_size=16))
    
    return evaluation

In [ ]:
image_size = 227

groups_folder_path = '../Data/AAFT_0/'

X_s = []

y = []
# auto_corr   hadamard     midAng    striping_color   striping_bw     striping_midAng_color     striping_midAng_bw
for label in ['Bi', 'Tri', 'Hammer', 'Rvcurl']:

    for feature in ['striping_color']:

        for top, dir, f in os.walk(groups_folder_path + label + "/" + feature + "/"):
            # print(f)
            for filename in f:
                # print(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.imread(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.resize(img, None, fx=image_size / img.shape[0], fy=image_size / img.shape[1])

                X_s.append(img / 256)

                if label == 'Bi':
                    y.append(0)
                elif label =='Tri':
                    y.append(1)
                elif label == 'Hammer':
                    y.append(2)
                else:
                    y.append(3)

print(X_s[0].shape)
# print(X_s)
print(len(y))
X = np.array(X_s)
y = np.array(y)

ins = []
for i in range(2):
    ins.append(inceptionV4_model(X, y))

(227, 227, 3)
239
171
Epoch 1/10
11/11 [==============================] - 53s 3s/step - loss: 1.5938 - accuracy: 0.1728 - val_loss: 7588.5601 - val_accuracy: 0.2500
Epoch 2/10
11/11 [==============================] - 2s 215ms/step - loss: 1.4381 - accuracy: 0.3482 - val_loss: 1595.3013 - val_accuracy: 0.2500
Epoch 3/10
 5/11 [============>.................] - ETA: 1s - loss: 1.2601 - accuracy: 0.4279

In [ ]:
for data in ins:
    print(data[0][0])

In [ ]:
for data in ins:
    print(data[1][1])

In [ ]:
'''
groups_folder_path = '../Data/resultVecsFigs/AAFT_4/'
X_s = []
y = []
# auto_corr   hadamard     midAng    striping_color   striping_bw     striping_midAng_color     striping_midAng_bw
for label in ['Bi', 'Tri']:

    for feature in ['hadamard']:

        for top, dir, f in os.walk(groups_folder_path + label + "/" + feature + "/"):
            # print(f)
            for filename in f:
                # print(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.imread(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.resize(img, None, fx=image_size / img.shape[0], fy=image_size / img.shape[1])

                X_s.append(img / 256)

                if label == 'Bi':
                    y.append(0)
                else:
                    y.append(1)

print(X_s[0].shape)
# print(X_s)
print(len(y))
X = np.array(X_s)
y = np.array(y)

ins = []
for i in range(10):
    ins.append(inceptionV4_model(X, y))

for data in ins:
    for d in data:
        for j in d:
            print(j)
'''